In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from config import Config
from depth_to_normals_tests import *
from img_utils import show_normals_components

# import cv2 as cv
# from scene_info import read_cameras, CameraEntry
# from dataclasses import dataclass
# from depth_to_normals import compute_normals_simple_diff_convolution_simple


In [2]:
def test_depth_to_normals(old_implementation=True, impl="svd"):

    planes_coeffs = np.array([
        [0, 0, 1, -1],
        [0, 0, 1, -100],
        [1, 0, 1, -1],
        [1, 0, 1, -100000],
        [0, 1, 1, -1],
        [1, 2, 2, -1],
        [1, 3, 3, -1],
        [1, 1, 1, -1],
        [2, 1, 2, -1],
        [3, 1, 3, -1],
    ])

    cameras = read_cameras("scene1")
    first_camera = next(iter(cameras.values()))

    for plane in planes_coeffs:

        #plane = -plane

        exact = plane[:3].copy()
        exact = -exact / np.linalg.norm(exact)
        print("\n\n\n\nTesting synthetic depth map for plane coeffs (ax + by + cz + d = 0): {}".format(plane))

        dsd = DepthSyntheticData(plane=plane, camera=first_camera, file_dir_and_name=get_file_dir_and_name(plane))
        depth_map_of_plane(dsd, allow_and_nullify_negative_depths=False, save=True)

        mask = torch.tensor([[0.5, 0, -0.5]]).float()

        if impl == "svd":
            depth, normals, clustered_normals, normal_indices = \
                compute_normals_from_svd(dsd.camera,
                                         dsd.file_dir_and_name[0],
                                         dsd.file_dir_and_name[1],
                                         save=False,
                                         output_directory=dsd.file_dir_and_name[0],
                                         )

        else:
            depth, normals, clustered_normals, normal_indices = \
                compute_normals_simple_diff_convolution_simple(dsd.camera,
                                                               dsd.file_dir_and_name[0],
                                                               dsd.file_dir_and_name[1],
                                                               save=False,
                                                               output_directory=dsd.file_dir_and_name[0],
                                                               override_mask=mask,
                                                               old_implementation=old_implementation
                                                               )


        normals_diff = normals - dsd.plane[:3]
        show_normals_components(normals_diff, "difference from exact result", (30.0, 20.0))

        if len(normals.shape) == 5:
            normals = normals.squeeze(dim=0).squeeze(dim=0)

        normals_np = normals.numpy()[5:-5, 5:-5]
        diff = normals_np - exact


        maxima = np.max(normals_np, axis=(0, 1))
        minima = np.min(normals_np, axis=(0, 1))
        max_dev = maxima - exact
        min_dev = minima - exact

        print("exact normal of the plane (x, y, y): {}".format(exact))
        print("normal component maxima (x, y, y): {}".format(maxima))
        print("normal component minima (x, y, y): {}".format(minima))
        print("difference of normal component maxima from the exact values (x, y, y): {}".format(max_dev))
        print("difference of normal component minima from the exact values (x, y, y): {}".format(min_dev))



In [3]:
Config.config_map[Config.show_normals_in_img] = False
test_depth_to_normals(old_implementation=False, impl="svd")





Testing synthetic depth map for plane coeffs (ax + by + cz + d = 0): [ 0  0  1 -1]



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

clustering normals starting
clustering normals finished. It took 0.6255159378051758
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: 0.0 degrees


...

clustering normals starting
clustering normals finished. It took 0.6181809902191162
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: nan degrees


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

exact normal of the plane (x, y, y): [-0.6882472  -0.22941573 -0.6882472 ]
normal component maxima (x, y, y): [-0.6882391  -0.22940344 -0.6882386 ]
normal component minima (x, y, y): [-0.6882557  -0.22942737 -0.6882557 ]
difference of normal component maxima from the exact values (x, y, y): [8.10401647e-06 1.22976866e-05 8.58085363e-06]
difference of normal component minima from the exact values (x, y, y): [-8.52567942e-06 -1.16335782e-05 -8.52567942e-06]


In [4]:
Config.config_map[Config.show_normals_in_img] = False
test_depth_to_normals(old_implementation=False, impl="diff")





Testing synthetic depth map for plane coeffs (ax + by + cz + d = 0): [ 0  0  1 -1]
Creating dir (if not exists already): work/tests/


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

clustering normals starting
clustering normals finished. It took 0.6741111278533936
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: 6.347489029197372 degrees


...

clustering normals starting
clustering normals finished. It took 0.32593417167663574
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: 23.469054205182612 degrees


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

exact normal of the plane (x, y, y): [-0.6882472  -0.22941573 -0.6882472 ]
normal component maxima (x, y, y): [-0.61175031 -0.10310398 -0.68861128]
normal component minima (x, y, y): [-0.69039718 -0.3431971  -0.73034478]
difference of normal component maxima from the exact values (x, y, y): [ 0.07649689  0.12631175 -0.00036408]
difference of normal component minima from the exact values (x, y, y): [-0.00214997 -0.11378136 -0.04209758]


In [5]:
Config.config_map[Config.show_normals_in_img] = False
test_depth_to_normals(old_implementation=True, impl="diff")





Testing synthetic depth map for plane coeffs (ax + by + cz + d = 0): [ 0  0  1 -1]
Creating dir (if not exists already): work/tests/


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

clustering normals starting
clustering normals finished. It took 0.2844579219818115
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: 19.714772425993402 degrees


...

clustering normals starting
clustering normals finished. It took 0.6746907234191895
show loc: False


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

angle between normal 0 and 1: 19.36066809866326 degrees


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

exact normal of the plane (x, y, y): [-0.6882472  -0.22941573 -0.6882472 ]
normal component maxima (x, y, y): [-0.0657701   0.03468002 -0.55515234]
normal component minima (x, y, y): [-0.7522776  -0.35480176 -0.99783021]
difference of normal component maxima from the exact values (x, y, y): [0.6224771  0.26409575 0.13309486]
difference of normal component minima from the exact values (x, y, y): [-0.0640304  -0.12538603 -0.30958301]
